In [12]:
import requests
import os
import re

In [221]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import sys
import shutil

In [358]:
from termcolor import cprint
colors = ['red', 'blue', 'yellow', 'green']

In [359]:
def Print(text, color):
    """ Prints `text` in passed `color` """
    cprint(text, color)

### LOG IN METHOD

In [290]:
def login(driver, username, password):        
    try: 
        username_input = driver.find_element_by_xpath('//input[@name = "username"]')
    except Exception:
        Print('Unable to find the username field.', colors[0])
        sys.exit()

    try: 
        password_input = driver.find_element_by_xpath('//input[@name = "password"]')
    except Exception:
        Print('Unable to find the password field.', colors[0])
        sys.exit()
    
    # sending the credentials
    try:
        sleep(10)
        username_input.send_keys(username)
        password_input.send_keys(password)
    except Exception:
        Print('Please check your connection and try again.', colors[0])
        sys.exit()

    try:
        sleep(5) # introduce sleep time as per your internet connection as to give the time to login page to load
        # login_link = driver.find_element_by_link_text('Log in')
        login_link = driver.find_element_by_css_selector("button[type='submit']")
        login_link.click()
    except Exception:
        Print('Unable to find the Login button.', colors[0])
        sys.exit()

    Print('Logging in...', colors[1])
    
    
    # check if the login was successful
    try:
        WebDriverWait(driver, 20).until(EC.title_is('Instagram'))
    except Exception:
        Print('Please try again with correct credentials or check your connection.', colors[0])
        sys.exit()

    Print('Login Successful!', colors[-1])

In [291]:
# def close_dialog_box(driver):
#     ''' Close the Notification Dialog '''
#     try:
#         close_btn = driver.find_element_by_xpath('//button[text()="Not Now"]')
#         close_btn.click()
#     except Exception:
#         pass 

### OPEN TARGET ACC

In [292]:
def open_target_profile(driver, main_url, target_username):  
    target_profile_url  = main_url + '/' + target_username
    sleep(10)
    Print('Redirecting to {0} profile...'.format(target_username), colors[1])
    
    # check if the target user profile is loaded. 
    try:
        driver.get(target_profile_url)
        WebDriverWait(driver, 20).until(EC.title_contains(target_username))
        sleep(10)
    except TimeoutError:
        Print('Some error occurred while trying to load the target username profile.', colors[0])
        sys.exit()

### Create Dir

In [366]:
# # check if the directory to store data exists
def create_dir(base_path, imagesData_path, descriptionsData_path, videoData_path):
    if not os.path.exists('data'):
        os.mkdir('data')
    if not os.path.exists(base_path):
        os.mkdir(base_path)
    if not os.path.exists(imagesData_path):
        os.mkdir(imagesData_path)
    if not os.path.exists(videoData_path):
        os.mkdir(videoData_path)
    if not os.path.exists(descriptionsData_path):
        os.mkdir(descriptionsData_path)

### Enter Account Info and Target Data

In [293]:
USERNAME = '******' 
PASSWORD = '*******'
TARGET_USERNAME = '**********'
MAIN_URL = 'https://www.instagram.com'

base_path = os.path.join('data', TARGET_USERNAME)
imagesData_path = os.path.join(base_path, 'images')
videoData_path = os.path.join(base_path, 'videos')
descriptionsData_path = os.path.join(base_path, 'descriptions')
create_dir(base_path, imagesData_path, descriptionsData_path, videoData_path)

### MAIN CELL

In [ ]:
# check the internet connection and if the home page is fully loaded or not. 
driver = webdriver.Chrome('chromedriver')

try:
    driver.get(MAIN_URL)
    WebDriverWait(driver, 20).until(EC.title_is('Instagram'))
    sleep(6)
except TimeoutError:
    Print('Loading took too much time. Please check your connection and try again.', colors[-2])
    sys.exit()

login(driver=driver, username=USERNAME, password=PASSWORD)
# close_dialog_box()
sleep(6)
open_target_profile(driver=driver, main_url=MAIN_URL, target_username=TARGET_USERNAME)

# count of posts, followers and following  "//ul[@class='_aa_7']/li[i]"
post_count = driver.find_element_by_xpath("//ul[@class='_aa_7']/li[1]").text
post_count = int(re.sub("\D", '', post_count))
followers_count = driver.find_element_by_xpath("//ul[@class='_aa_7']/li[2]").text
following_count = driver.find_element_by_xpath("//ul[@class='_aa_7']/li[3]").text

mostly_data = dict({
    "posts": post_count,
    "follower":followers_count,
    "following":following_count,
})

descriptions=[]
image_list=[]
video_list=[]

for post in range(post_count):
    if post == 0:
        # 2. open first post 
        first_post = driver.find_element_by_xpath("//article/div[1]/div/div[1]/div[1]/a")
        first_post.click()
        sleep(10)

    # 3.1. if is video
    video_button = driver.find_elements_by_xpath("//button[@aria-label='Toggle audio'][@class='_acan _acao _acas']")

    # 3.2. image or video
    if video_button.__len__():
        video = driver.find_element_by_xpath("//div[@class='_acam _ab1n']/div[@class='_ab1c']/video").get_attribute('src')
        video_list.append(video)
        print(f"VVVVV Is Video VVVV:  {video}")
    else:
        img = driver.find_element_by_xpath("//div[contains(@class,'_aagu ')]/div[@class='_aagv']/img").get_attribute('src')
        image_list.append(img)
        print(f"IIII Is Image IIII: {img}")
    
    # 3.3. description  
    desc = driver.find_element_by_xpath("//div[@class='_a9zm']/div[@class='_a9zn _a9zo']/div[@class='_a9zr']/div[@class='_a9zs']/span").text
    descriptions.append(desc)

    # 3.4. next post button 
    next_post = driver.find_element_by_xpath("//div[@class='_aani _aanj']/div[@class='_aank']/div[@class=' _aaqg _aaqh']/button")
    next_post.click()
    sleep(10)
    Print(f'====>> Go to post: {post+1} <<====', colors[-2])


### Clean Description

In [308]:
new_descriptions = []
for i, t in enumerate(descriptions):
    new_descriptions.append(re.sub("\n.", "" , t))

### GET Account INFO

In [309]:
mostly_data

{'posts': 1420, 'follower': '157K followers', 'following': '580 following'}

### SAVE DATA

In [367]:
def save_description(descriptions_list):
    with open(descriptionsData_path+'/description_file.txt', 'wb') as file:
        for i, line in enumerate(descriptions_list):
            file.write((str(i+1) + '.' + line+ '\n').encode())

In [368]:
def download_posts(image_list):
    ''' To download all the posts of the target account ''' 

    no_of_images = len(image_list)

    for index, link in enumerate(image_list, start = 1):
        filename = 'image_' + str(index) + '.jpg'
        image_path = os.path.join(imagesData_path, filename)

        response = requests.get(link, stream=True)
        Print('Downloading image {0} of {1}'.format(index, no_of_images), colors[-1])
        try:
            with open(image_path, 'wb') as file:
                shutil.copyfileobj(response.raw, file)
        except Exception as e:
            Print(e, colors[0])
            Print('Couldn\'t download image {0}.'.format(index), colors[0])
            Print('Link for image {0} ---> {1}'.format(index, link), colors[0])
    Print('Download completed!', colors[-1])

### Save Images 

In [362]:
download_posts(image_list)

Download completed!


### Save Description of posts

In [369]:
save_description(new_descriptions)

### Close Driver

In [ ]:
driver.close()

## $$ Thank~~You $$
### $$ Salim~Fathy $$